### アイデア
ガウス分布で回帰する。

出力にパラメトライゼーショントリックを用いて、サンプルされたzを出力するようにする。
そのzがなるべく次時刻の真の値と近いものになるようにすれば、分散込みで学習できるのでは？というアイデア

いろいろ工夫してみたがだめだった

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime

### データの読み込み

In [ ]:
def load_data():
    """
    return X_train, X_test, y_train, y_test
    """
    print("loading the data...")
    DATA_DIR="./data/"
    X_train = np.load(os.path.join(DATA_DIR, "X_train.npy"))
    print("X train",X_train.shape)
    X_test = np.load(os.path.join(DATA_DIR, "X_test.npy"))
    print("X test",X_test.shape)
    y_train = np.load(os.path.join(DATA_DIR, "y_train.npy"))
    print("y train",y_train.shape)
    y_test = np.load(os.path.join(DATA_DIR, "y_test.npy"))
    print("y test",y_test.shape)
    
    # shapeをglobal変数に
    global NUM_timesteps, NUM_input_dim, NUM_output_dim
    _, NUM_timesteps, NUM_input_dim = X_train.shape
    _, NUM_output_dim = y_train.shape
    
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = load_data()

### モデルの作成

In [ ]:
from keras.layers import Dense, RepeatVector, Input, Lambda
from keras.layers import GRU
# from keras.layers import CuDNNGRU as GRU #GPU用
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.optimizers import adam
from keras import backend as K
from keras.models import Model

In [ ]:
def ret_model():
    """
    GRU
    GRU
    FC
    FC
    ↓  ↓
    mu sigma
    
    oをサンプリング
    
    というモデルを返す
    """
    def sampling(args):
        """
        z_mean, z_log_var=argsからoをサンプリングする関数
        戻り値
            o (tf.tensor):サンプリングされた結果
        """
        z_mean, z_log_var = args
        # いきなり標準偏差を求めてしまっても構わないが、負を許容してしまうのでこのようなトリックを用いている
        batch_size = K.shape(z_mean)[0]
        eps = K.random_normal(shape=(batch_size,100))
        return z_mean + K.exp(0.5*z_log_var)*eps   
    
    
    # hyper parameter
    LATENT = 20
    FC=20
    
    # ネットワークの定義
    inputs = Input(shape=(NUM_timesteps, NUM_input_dim))
    # (, NUM_timesteps, NUM_input_dim)
    gru=GRU(LATENT, return_sequences=True)(inputs)
    # (, NUM_timesteps, LATENT)
    gru=GRU(LATENT)(gru)
    # (, LATENT)
    fc=Dense(FC)(gru)
    # (, FC)
    o_mean=Dense(NUM_output_dim,name="mean")(fc)
    o_log_var=Dense(NUM_output_dim,name="log_var")(fc)
    # 双方とも(, NUM_output_dim)
    output = Lambda(sampling, output_shape=(100,),name="output")\
    ([o_mean,o_log_var])
    # (,100)
    
    model = Model(inputs,output)    
    
    model.summary()
    
    return model       

In [ ]:
model = ret_model()
# def loss(y_true, pred):
#     """
#     predは2つの出力を持つ
#     """
#     mean, log_var = pred[0], pred[1]
#     sigma=K.exp(0.5*log_var)
#     loss = 0.5+K.log(2*np.pi*sigma)+(y_true-mean)/(np.sqrt(2)*sigma)
#     #勾配消失してしまう
#     return K.mean(loss)
# def loss(y_true, pred):
#     return K.square(pred[1]-2*K.square(y_true-pred[0]))
model.compile(optimizer=adam(lr=0.001),loss="mean_squared_error")

### 訓練

In [ ]:
logdir="./"+str(datetime.now().strftime('%s'))+"model/"
model.fit(X_train,np.tile(y_train,100),
         epochs=20,
         batch_size=128,
         shuffle=True,
         validation_split=0.1,
         callbacks=[TensorBoard(log_dir=logdir), 
                   EarlyStopping(patience=2),
                   ModelCheckpoint(filepath = logdir+'model_epoch.{epoch:02d}-los{val_loss:.4f}.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')])

### 結果の可視化

In [ ]:
predictor=K.function([model.input],[model.get_layer("mean").output, model.get_layer("log_var").output])

In [ ]:
mu_pred, log_var_pred = predictor([X_test])
std_dev = np.exp(0.5*log_var_pred)

In [ ]:
upper = mu_pred+std_dev
lower = mu_pred-std_dev
upper.shape, lower.shape

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(mu_pred[:192])
plt.fill_between(range(192),upper[:192,0],lower[:192,0],color="green",alpha=0.2)
plt.plot(y_test[:192])

In [ ]:
plt.plot(std_dev[:192])